In [1]:
import pandas as pd
import os

#Read the csv from Original Downloads. Note: Replica website offers much more info (columns) than here. To reduce the file size, a subset of columns were only downloaded
# For next time maybe look into for each file iterating over the folder #equivalent to batch scripting on Python                                                                                                        1
#Spring2023
#(Had to download Weekend data on 3-15-2024 because I did not include "previous trip purpose" in 02-22-2024 download)
#file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekends-03_15_24-trips_dataset.csv"
#df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekends-03_15_24-trips_dataset.csv")
file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv"
df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv")

#sample = df.sample(1000)
#taking a sample
#df = df_Org.sample(1000)

#Fall2022
#file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekend-02_22_24-trips_dataset.csv"
#df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekend-02_22_24-trips_dataset.csv")
#file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekday-02_22_24-trips_dataset.csv"
#df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekday-02_22_24-trips_dataset.csv")           #takes ~1min

print("The shape of original download file is df-", df.shape)




C:\Users\USAP093587\AppData\Local\Temp\ipykernel_18564\3536400706.py:11: DtypeWarning: Columns (16,17,39,40,41,48,49,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv")


The shape of original download file is df- (8510893, 52)


In [ ]:
# Taking a subset of the columns that we need
subset_columns= ['origin_trct_2020', 'origin_trct_fips_2020', 'origin_cty_fips_2020', 'origin_cty_2020',                                      
                 'destination_trct_2020', 'destination_trct_fips_2020', 'destination_cty_fips_2020', 'destination_cty_fips_2020',
                 'primary_mode', 'trip_purpose', 'previous_trip_purpose',                                                                      #previous trip purpose
                 'trip_start_time',  'trip_distance_miles', 'trip_duration_minutes', 
                 'trip_taker_household_income', 
                 'trip_taker_work_trct_2020', 'trip_taker_work_cty_2020','trip_taker_work_trct_fips_2020',
                 'trip_taker_home_trct_2020', 'trip_taker_home_cty_2020',  'trip_taker_home_trct_fips_2020', 
                ]
sub_df = df[subset_columns] 
print("After taking the immediate subset of columns, The shape of this file is", sub_df.shape)

## Filtering the counties
filtered_df = (sub_df[(sub_df['trip_taker_work_cty_2020'].isin(['Pierce', 'King', 'Snohomish','Kitsap'])) &                                          # Update this on .py
                      (sub_df['trip_taker_home_cty_2020'].isin(['Pierce', 'King', 'Snohomish','Kitsap']))])
print("After filtering trip taker's work home & office within Pierce, King, Snohomish, Kitsap, shape is",filtered_df.shape)

# Checking
print("Trip taker's work", filtered_df['trip_taker_work_cty_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_work_cty_2020'].isna().sum() )
print("Trip taker's home",filtered_df['trip_taker_home_cty_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_home_cty_2020'].isna().sum() )



In [ ]:
# Rounding time to the nearest Hour
filtered_df['trip_start_time'] = pd.to_datetime(filtered_df['trip_start_time']).dt.round('H').dt.hour

import numpy as np
#Rounding the trip distance
filtered_df['trip_distance_miles'] = filtered_df['trip_distance_miles'].apply(np.ceil)

print("Rows filtered for Orign, Destination in Replica & trip_taker's home and office here in this code", filtered_df.shape)



In [ ]:
#filtered_df[ 'trip_taker_work_trct_fips_2020'].shape
#df[ 'trip_taker_work_trct_fips_2020'].unique
#filtered_df.loc[filtered_df['trip_taker_work_trct_fips_2020'] == "Does not have work/school location"]      #1_718_423 from df #0 from filtered

In [ ]:
# Some trips does not have Trip taker's work and/or home location information. For these ones, FIPS tract values are string. 
# Now deleting all non-numeric rows from trip_taker_work_trct_fips_2020 & other as necessary
# Big Learning here .loc doesnt change the data type
filtered_df[ 'trip_taker_work_trct_fips_2020'] = pd.to_numeric(filtered_df['trip_taker_work_trct_fips_2020'], errors='coerce').astype('Int64')
filtered_df['trip_taker_home_trct_fips_2020'] = pd.to_numeric(filtered_df['trip_taker_home_trct_fips_2020'], errors='coerce').astype('Int64')
filtered_df['origin_trct_fips_2020'] = pd.to_numeric(filtered_df['origin_trct_fips_2020'], errors='coerce').astype('Int64')

print("Trip taker's home",filtered_df['trip_taker_home_trct_fips_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_home_trct_fips_2020'].isna().sum() )
print("Trip taker's work",filtered_df['trip_taker_work_trct_fips_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_work_trct_fips_2020'].isna().sum() )
print("Original code dropped all empty values here")
# Should I drop all "Nan" rows? Maybe at the end
#filtered_df.dropna(subset=['trip_taker_work_trct_fips_2020'], inplace=True)                                                 #didnt do much for spring23 weekday Original code drops this

print("The shape is now", filtered_df.shape)                   #from ReplicaOrg= (7885167, 53)>> (5167903, 20) >> (5167903, 20)
filtered_df["trip_taker_home_trct_fips_2020"].value_counts()
print("Trip taker tract has", filtered_df["trip_taker_home_trct_fips_2020"].isna().sum(), "empty values")



In [ ]:
#Checking if any empty values in 'trip_taker_household_income_bin
#print("The empty values in Trip taker HH income is ",filtered_df["trip_taker_household_income"].isnull().sum())  

# Converting income to bins
# 1 -- Less than or equal to 25000
# 2 -- More than 25000 and less than or equal to 50000
# 3 -- More than 50000 and less than or equal to 75000
# 4 -- More than 75000 and less than or equal to 100000
# 5 --More than 100000
# Define the bin edges and labels
bin_edges = [-float('inf'), 0, 25000, 50000, 75000, 100000, float('inf')]        #Note: There are some -ve income which is converted to Bin1
bin_labels = [1, 1, 2, 3, 4, 5]
# Use pd.cut to categorize the column values
filtered_df.loc[:,'trip_taker_household_income_bin'] = pd.cut(filtered_df['trip_taker_household_income'], bins=bin_edges, labels=bin_labels, right=False, ordered=False)#.fillna('9)')                            #change in .py NA-changes


# Checking
#filtered_df["trip_taker_household_income"].describe()        #has negative values too
#filtered_df["trip_taker_household_income"].info()            #Confirmed 0 empty
#df=filtered_df
#df[df["trip_taker_household_income_bin"].isnull()][['trip_taker_household_income_bin', 'trip_taker_household_income']].info()
#df[df["trip_taker_household_income_bin"].isnull()][['trip_taker_household_income_bin', 'trip_taker_household_income']].info()   664 rows

#filtered_df["trip_taker_household_income_bin"].unique()



In [ ]:
Subset_for_mapping = filtered_df.copy()
# To Map tracts to Travelshed                                                                                                                                                                                    2 Dict Vs Join
# Get the mapping file
TravelShed_w_all_FIPS = pd.read_csv(f"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/TravelShed_FIPS.csv")
# This File is located in Sharepoint with block group code(bg_code) and its corresponding travelshed. Took out the last digit to get Tract FIPS and use it to map.   

# Put csv in the dictionary
TravelShed_dict = dict(zip(TravelShed_w_all_FIPS['tract'], TravelShed_w_all_FIPS['travelshed']))

#Mapping tract from original dataframe to Travelshed to new columns
Subset_for_mapping['Origin_sline_shed'] = Subset_for_mapping['origin_trct_fips_2020'].map(TravelShed_dict).fillna(99)#, inplace=True) 
Subset_for_mapping['Destination_sline_shed'] = Subset_for_mapping['destination_trct_fips_2020'].map(TravelShed_dict).fillna(99)#, inplace=True)
Subset_for_mapping['Trip_taker_home_sline_shed'] = Subset_for_mapping['trip_taker_home_trct_fips_2020'].map(TravelShed_dict).fillna(99)#, inplace=True)
Subset_for_mapping['Trip_taker_work_sline_shed'] = Subset_for_mapping['trip_taker_work_trct_fips_2020'].map(TravelShed_dict).fillna(99)#,inplace=True)

#Checking if location and TravelShed mapping went OK
#Subset_for_mapping[['Origin_sline_shed' ,'origin_trct_fips_2020', 'Destination_sline_shed' ,'destination_trct_fips_2020']].head(10)
#Subset_for_mapping[['Trip_taker_home_sline_shed' ,'trip_taker_home_trct_fips_2020', 'trip_taker_home_trct_2020', 'Trip_taker_work_sline_shed' ,'trip_taker_work_trct_fips_2020', 'trip_taker_work_trct_2020']].value_counts()

print("The shape after Travel Sheds is",  Subset_for_mapping.shape)           # 4 travel sheds: 'Origin_sline_shed', 'Destination_sline_shed','Trip_taker_home_sline_shed', 'Trip_taker_work_sline_shed'
                                                                              #+ 1 Income bin: 'trip_taker_household_income_bin'


print("The trip taker's home Travel-Shed are",Subset_for_mapping["Trip_taker_home_sline_shed"].value_counts())
print("The trip taker's home Travel-Shed are",Subset_for_mapping["Trip_taker_work_sline_shed"].value_counts())
print("The trip Origin Travel-Shed are",Subset_for_mapping["Origin_sline_shed"].value_counts())
print("The trip Destination Travel-Shed are",Subset_for_mapping["Destination_sline_shed"].value_counts())




In [ ]:
#Subset_for_mapping.columns
#Subset_for_mapping.head(10)

subset_for_summary =Subset_for_mapping.copy()

# Start time >> Start hour
#subset_for_summary.rename(columns = {'trip_start_time':'trip_start_hour'}, inplace = True) 
# drop county code, trip duration, others
#subset_for_summary = subset_for_summary.drop(['origin_cty_fips_2020','origin_trct_2020','destination_trct_2020', 'destination_cty_fips_2020','trip_taker_home_trct_2020','trip_taker_home_cty_2020', 'trip_duration_minutes'],axis=1) #,'trip_duration_minutes' #'origin_cty_fips_2020','trip_taker_work_trct_2020.1',
print("The shape is now", subset_for_summary.shape)
print("The columns remaining are", subset_for_summary.columns)


In [ ]:

# Now categorizing these columns to numericals
#Primary-mode ['private_auto', 'auto_passenger', 'public_transit','other_travel_mode', 'on_demand_auto', 'biking', 'walking'], dtype=object)
# 1 'private_auto'
# 2  'auto_passenger'
# 3 'public_transit'
# 4 'on_demand_auto'
# 5 'biking'
# 6 'walking'
# 7 'other_travel_mode'

#Trip purpose 'home', 'work', 'shop', 'social', 'eat', 'other_activity_type', 'maintenance', 'school', 'recreation'
#1 'home'
#2 'work'
#3 'shop'
#4 'social'
#5 'eat'
#6 'school'
#7 'recreation'
#8 'maintenance'
#9 'other_activity_type'

#TravelShed

#Origin_sline_shed =           ['99', 'Tukwila', 'Auburn', 'Kent', 'Sumner', 'Lakewood', 'Puyallup', 'King Street', 'Tacoma Dome', 'South Tacoma'],dtype=object)
#'Destination_sline_shed'=     ['99', 'Kent', 'King Street', 'Tukwila', 'Puyallup', 'Sumner',  'Tacoma Dome', 'Auburn', 'Lakewood', 'South Tacoma'], dtype=object)
#'Trip_taker_home_sline_shed'= ['99', 'Kent', 'Tacoma Dome', 'Sumner', 'Lakewood', 'Puyallup','Auburn', 'King Street', 'Tukwila', 'South Tacoma'], dtype=object)
#'Trip_taker_work_sline_shed'= ['99', 'King Street', 'Tukwila', 'Kent', 'Auburn', 'Tacoma Dome', 'Puyallup', 'Sumner', 'Lakewood', 'South Tacoma'], dtype=object) 
#df['trip_purpose'].unique() 

# Categorizing a few columns to reduce the file size
# Creating an empty Dictionary
Dict = {}
Shed_list = [('Auburn',1), ('Kent',2), ('King Street',3), ('Lakewood',4), ('Puyallup',5), ('South Tacoma',6), ('Sumner',7), ('Tacoma Dome',8), ('Tukwila',9), (99,99)]
Mode_list = [('private_auto',1), ('auto_passenger',2), ('public_transit',3), ('on_demand_auto',4), ('biking',5), ('walking',6),('other_travel_mode',7)]
Trp_purpose_list= [('home',1), ('work',2), ('shop',3), ('social',4), ('eat',5), ('school',6), ('recreation',7), ('maintenance',8), ('other_activity_type',9)]
 
# Creating a Dictionary # with each item as a Pair
Shed_Dict = dict(Shed_list)
Mode_Dict = dict(Mode_list)
Trp_purpose_Dict= dict(Trp_purpose_list)
#print(Dict)


#Now Mapping
subset_for_summary['Origin_sline_shed_numeric'] = subset_for_summary['Origin_sline_shed'].map(Shed_Dict) 
subset_for_summary['Destination_sline_shed_numeric'] = subset_for_summary['Destination_sline_shed'].map(Shed_Dict) 
subset_for_summary['Trip_taker_home_sline_shed_numeric'] = subset_for_summary['Trip_taker_home_sline_shed'].map(Shed_Dict) 
subset_for_summary['Trip_taker_work_sline_shed_numeric'] = subset_for_summary['Trip_taker_work_sline_shed'].map(Shed_Dict) 

subset_for_summary['Primary_Mode_numeric'] = subset_for_summary['primary_mode'].map(Mode_Dict) 
subset_for_summary['Trip_purpose_numeric'] = subset_for_summary['trip_purpose'].map(Trp_purpose_Dict)

# checking the map 
subset_for_summary [['Origin_sline_shed','Origin_sline_shed_numeric','Destination_sline_shed', 'Destination_sline_shed_numeric','Trip_taker_home_sline_shed','Trip_taker_home_sline_shed_numeric','Trip_taker_work_sline_shed','Trip_taker_work_sline_shed_numeric','primary_mode','Primary_Mode_numeric','trip_purpose','Trip_purpose_numeric']]

subset_for_summary[['Origin_sline_shed','Origin_sline_shed_numeric','Destination_sline_shed', 'Destination_sline_shed_numeric','Trip_taker_home_sline_shed','Trip_taker_home_sline_shed_numeric','Trip_taker_work_sline_shed','Trip_taker_work_sline_shed_numeric','primary_mode','Primary_Mode_numeric','trip_purpose','Trip_purpose_numeric']].tail(10)
print(subset_for_summary["Origin_sline_shed"].value_counts())



## Finding HBW trips only

In [ ]:
#Now, trying to get only HBW trips & its hourly counts
#For HBW Columns
HBW_col = ['trip_purpose','previous_trip_purpose' ]

# Concatenate values of all columns to create a new column 'concatenated'
subset_for_summary['HBW_concat'] = subset_for_summary[HBW_col].agg('-'.join, axis=1)

#subset_for_summary[['trip_purpose','previous_trip_purpose','HBW_concat','trip_start_time']]
subset_for_summary =  subset_for_summary[subset_for_summary['HBW_concat'].str.contains('work-home|home-work')] 
print("The HBW trips table's size is", subset_for_summary.shape)


In [ ]:
# Intermediary file writeup, before making trip counts for each matching rows
file_name = os.path.basename(file_path).replace(".csv", "_afterNumeric.csv")
file_path = os.path.join("C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final", file_name).replace("\\","/")
subset_for_summary.to_csv(file_path, index=False)

#Since mapping is ok
drop_col = ['trip_taker_work_cty_2020','Origin_sline_shed','Destination_sline_shed', 'Trip_taker_home_sline_shed','Trip_taker_work_sline_shed','primary_mode', 'trip_purpose']
subset_after_numeric = subset_for_summary.drop(columns=drop_col, axis=1)



In [ ]:
subset_after_numeric = subset_for_summary    #Comment it later
#Lets rename the numeric columns & others
subset_after_numeric = subset_after_numeric.rename(columns={'Origin_sline_shed_numeric': 'origin_travel_shed', 
                        'Destination_sline_shed_numeric': 'destination_travel_shed',
                        'Trip_taker_home_sline_shed_numeric':'trip_taker_home_travel_shed',
                        'Trip_taker_work_sline_shed_numeric': 'trip_taker_work_travel_shed',
                        'Primary_Mode_numeric':'primary_mode',
                        'Trip_purpose_numeric':'trip_purpose',
                        'trip_start_time':'trip_start_hour'
                        })

print("Now the shape is", subset_after_numeric.shape)
print("The columns are", subset_after_numeric.columns)

final_subset = subset_after_numeric[['origin_trct_fips_2020','destination_trct_fips_2020','primary_mode', 'trip_purpose','previous_trip_purpose','trip_start_hour','trip_distance_miles','trip_taker_work_trct_fips_2020','trip_taker_home_trct_fips_2020','trip_taker_household_income_bin','origin_travel_shed',
       'destination_travel_shed', 'trip_taker_home_travel_shed','trip_taker_work_travel_shed', 'HBW_concat' ]]



## Making the trip counts

In [ ]:
#final_subset
# Concatenate values of all columns to create a new column 'concatenated'
final_subset['concatenated'] = final_subset[final_subset.columns].astype(str).agg('-'.join, axis=1)

# Create a new column 'counts' by counting occurrences of each unique combination
final_subset['trip_counts'] = final_subset.groupby('concatenated')['concatenated'].transform('count')        # I could use trip counts

# Drop the 'concatenated' column if not needed anymore
final_subset = final_subset.drop(columns=['concatenated'])
#final_subset



In [ ]:
#final_subset.shape    #(1409494, 18)
#final_subset['trip_taker_work_travel_shed'].value_counts()
#843931/1_409_494


In [ ]:
#hourly_counts=final_subset.groupby('trip_start_hour').size()

#To minimize the file size # I should have done much earlier
cols_toInt8 = ['primary_mode', 'trip_purpose','trip_start_hour','trip_distance_miles','origin_travel_shed','destination_travel_shed','trip_taker_home_travel_shed','trip_taker_work_travel_shed','trip_counts']
final_subset[cols_toInt8] = final_subset[cols_toInt8].applymap(np.int8)

# Writing final_subset
#Subset_before_merged = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset"
#file_path = # already given above while reading original replica download
file_name = os.path.basename(file_path).replace("_afterNumeric.csv", "_subset.csv")
file_path = os.path.join("C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/Final_Subset", file_name).replace("\\","/")
final_subset.to_csv(file_path, index=False)



#Sample
file_name = os.path.basename(file_path).replace( "_afterNumeric.csv", ".csv")
path=os.path.join("C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/Sample_final","_FinalSubset_Sample.csv").replace("\\","/")
final_subset.sample(100).to_csv(path, index=False)

In [ ]:
#final_subset['HBW_concat'].unique()
final_subset.shape
final_subset.columns

## Testing

In [ ]:
# Checking trips between travelsheds

trips_within_travelshed = final_subset[(final_subset['origin_travel_shed'].isin([1,2,3,4,5,6,7,8,9,99])) & 
                                             (final_subset['destination_travel_shed'].isin([1,2,3,4,5,6,7,8,9]))]

In [ ]:
#trips_within_travelshed.shape
#trips_within_travelshed['origin_travel_shed'].value_counts()
#trips_within_travelshed['destination_travel_shed'].value_counts()
#trips_within_travelshed['trip_taker_home_travel_shed'].value_counts()
trips_within_travelshed['trip_taker_work_travel_shed'].value_counts()             #.isnull().sum()

In [ ]:
# Let me explore those external-to-Travel-Shed trips
trips_within_travelshed['trip_taker_work_travel_shed']
trips_originating_ExternalToTraveShed =  trips_within_travelshed[trips_within_travelshed['origin_travel_shed']==99] 
#trips_originating_ExternalToTraveShed.shape
#trips_originating_ExternalToTraveShed

#trips_originating_ExternalToTraveShed.groupby('origin_trct_fips_2020').size().sort_values(ascending=False) .head(10)    
trips_originating_ExternalToTraveShed.groupby('destination_trct_fips_2020').size().sort_values(ascending=False) .head(10)   
# Most trips seems to be coming from 
#origin_trct_fips_2020
#               TRIPS
# 53033022803    3546       Kings County
# 53033010900    2756       Kings County
# 53033028402    2411       Kings County
# 53053070703    1743       Snohomish County
# 53033027200    1737       Kings County


#hourly_counts=trips_within_travelshed.groupby('trip_start_hour').size()

In [ ]:
trips_within_travelshed

In [ ]:
# #For HBW Columns
# HBW_col = ['trip_purpose','previous_trip_purpose' ]

# #final_subset
# # Concatenate values of all columns to create a new column 'concatenated'
# subset_for_summary['HBW_concat'] = subset_for_summary[HBW_col].agg('-'.join, axis=1)

# subset_for_summary[['trip_purpose','previous_trip_purpose','HBW_concat','trip_start_time']]
# subset_for_summary =  subset_for_summary[subset_for_summary['HBW_concat'].str.contains('work-home|home-work')] 
# print("The HBW trips table's size is", subset_for_summary.shape)

hourly_counts=trips_within_travelshed.groupby('trip_start_hour').size()

import matplotlib.pyplot as plt
# Create a histogram
plt.figure(figsize=(10, 6))  # Optional: Set the figure size
plt.bar(hourly_counts.index, hourly_counts, color='skyblue', edgecolor='black')
plt.xlabel('Hour of the Day')
plt.ylabel('Count')
plt.title('Hourly Count of Trips')
plt.grid(axis='y')  # Optional: Add grid lines

# Show the plot
plt.show()

In [ ]:
3_641_088/6269415 
#+426_677+ 272_632+ 252_366+ 243_885+ 218_106+211_406+ 166_186+ 154_695+ 74_917

In [ ]:
final_subset['HBW_concat'].unique()

In [ ]:
subset_for_summary_HBW[['trip_purpose','previous_trip_purpose','HBW_concat','trip_start_time']]

In [ ]:
#filtered_df_HBW =  filtered_df[filtered_df['HBW_concat'].str.contains('work-home|home-work')]  #1000 > (165, 23)

subset_after_numeric.columns

In [ ]:
subset_after_numeric[['trip_start_hour','trip_purpose','previous_trip_purpose']]

In [ ]:
df.columns

In [ ]:
#filtered_df['previous_trip_purpose'].unique() #['eat', 'maintenance', 'recreation', 'work_from_home', 'school','social', 'work', 'home', 'shop', 'other_activity_type']
filtered_df['trip_purpose'].unique()           #[ 'eat', 'maintenance', 'recreation',                   'school','social', 'work', 'home', 'shop', 'other_activity_type']

In [ ]:
file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-03_14_24-trips_dataset.csv"
df_Org= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-03_14_24-trips_dataset.csv")
df_Org.columns

In [ ]:
file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv"
df_Org_Feb= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv")
df_Org_Feb.columns

# Code that takes subset of Replica data & filters HBW Trips only too

In [ ]:
# %%
# from scratch with 2/24 datasets
#Updated 3/15/2024

import pandas as pd
import os

#Read the csv from Original Downloads. Note: Replica website offers much more info (columns) than here. To reduce the file size, a subset of columns were only downloaded
# For next time maybe look into for each file iterating over the folder #equivalent to batch scripting on Python                                                                                                        1
#Spring2023
file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekends-03_15_24-trips_dataset.csv"
df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekends-03_15_24-trips_dataset.csv")
#file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv"
#df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/ReplicaOriginalDownload/replica-soundtransit_spring2023_weekday-02_22_24-trips_dataset.csv")

#sample = df.sample(1000)
#taking a sample
#df = df_Org.sample(1000)

#Fall2022
#file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekend-02_22_24-trips_dataset.csv"
#df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekend-02_22_24-trips_dataset.csv")
#file_path = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekday-02_22_24-trips_dataset.csv"
#df= pd.read_csv(r"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/DownloadsV3/replica-soundtransit_fall2022_weekday-02_22_24-trips_dataset.csv")           #takes ~1min

print("The shape of this file is", df.shape)

# Taking a subset of the columns that we need

subset_columns= ['origin_trct_2020', 'origin_trct_fips_2020', 'origin_cty_fips_2020', 'origin_cty_2020',                                      
                 'destination_trct_2020', 'destination_trct_fips_2020', 'destination_cty_fips_2020', 'destination_cty_fips_2020',
                 'primary_mode', 'trip_purpose', 'previous_trip_purpose',                                                                      #previous trip purpose
                 'trip_start_time',  'trip_distance_miles', 'trip_duration_minutes', 
                 'trip_taker_household_income', 
                 'trip_taker_work_trct_2020', 'trip_taker_work_cty_2020','trip_taker_work_trct_fips_2020',
                 'trip_taker_home_trct_2020', 'trip_taker_home_cty_2020',  'trip_taker_home_trct_fips_2020', 
                ]
sub_df = df[subset_columns] 
print("After taking the immediate subset of columns, The shape of this file is", sub_df.shape)

## Filtering the counties
#filtered_df = sub_df.loc[sub_df['trip_taker_work_cty_2020'].isin(['Pierce', 'King', 'Snohomish','Kitsap' ])]
#print("After filtering trip taker's work city within Pierce, King, Snohomish, Kitsap, shape is",filtered_df.shape)     #1000>>956

#filtered_df = sub_df.loc[sub_df['trip_taker_home_cty_2020'].isin(['Pierce', 'King', 'Snohomish','Kitsap' ])]         
#print("After filtering trip taker's work home within Pierce, King, Snohomish, Kitsap, shape is",filtered_df.shape)

filtered_df = (sub_df[(sub_df['trip_taker_work_cty_2020'].isin(['Pierce', 'King', 'Snohomish','Kitsap'])) &                                          # Update this on .py
                      (sub_df['trip_taker_home_cty_2020'].isin(['Pierce', 'King', 'Snohomish','Kitsap']))])
print("After filtering trip taker's work home & office within Pierce, King, Snohomish, Kitsap, shape is",filtered_df.shape)
print("Mind that trip data was downloaded where trips' origin & destination were in 5 neighbouring counties")

# Checking
print("Trip taker's work", filtered_df['trip_taker_work_cty_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_work_cty_2020'].isna().sum() )
print("Trip taker's home",filtered_df['trip_taker_home_cty_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_home_cty_2020'].isna().sum() )

# Rounding time to the nearest Hour
filtered_df['trip_start_time'] = pd.to_datetime(filtered_df['trip_start_time']).dt.round('H').dt.hour

import numpy as np
#Rounding the trip distance
filtered_df['trip_distance_miles'] = filtered_df['trip_distance_miles'].apply(np.ceil)

print("Rows filtered for Orign,Destination in Replica & trip_taker's home and office here in this code", filtered_df.shape)

# Some trips does not have Trip taker's work and/or home location information. For these ones, FIPS tract values are string. 
# Now deleting all non-numeric rows from trip_taker_work_trct_fips_2020 & other as necessary
# Big Learning here .loc doesnt change the data type
filtered_df[ 'trip_taker_work_trct_fips_2020'] = pd.to_numeric(filtered_df['trip_taker_work_trct_fips_2020'], errors='coerce').astype('Int64')
filtered_df['trip_taker_home_trct_fips_2020'] = pd.to_numeric(filtered_df['trip_taker_home_trct_fips_2020'], errors='coerce').astype('Int64')
filtered_df['origin_trct_fips_2020'] = pd.to_numeric(filtered_df['origin_trct_fips_2020'], errors='coerce').astype('Int64')

print("Trip taker's home",filtered_df['trip_taker_home_trct_fips_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_home_trct_fips_2020'].isna().sum() )
print("Trip taker's work",filtered_df['trip_taker_work_trct_fips_2020'].value_counts(), "& empty rows are",filtered_df['trip_taker_work_trct_fips_2020'].isna().sum() )
print("Original code dropped all empty values here")
# Should I drop all "Nan" rows? Maybe at the end
#filtered_df.dropna(subset=['trip_taker_work_trct_fips_2020'], inplace=True)                                                 #didnt do much for spring23 weekday Original code drops this

print("The shape is now", filtered_df.shape)                   #from ReplicaOrg= (7885167, 53)>> (5167903, 20) >> (5167903, 20)
filtered_df["trip_taker_home_trct_fips_2020"].value_counts()
print("Trip taker tract has", filtered_df["trip_taker_home_trct_fips_2020"].isna().sum(), "empty values")

#Checking if any empty values in 'trip_taker_household_income_bin
#print("The empty values in Trip taker HH income is ",filtered_df["trip_taker_household_income"].isnull().sum())  

# Converting income to bins
# 1 -- Less than or equal to 25000
# 2 -- More than 25000 and less than or equal to 50000
# 3 -- More than 50000 and less than or equal to 75000
# 4 -- More than 75000 and less than or equal to 100000
# 5 --More than 100000
# Define the bin edges and labels
bin_edges = [-float('inf'), 0, 25000, 50000, 75000, 100000, float('inf')]
bin_labels = [1, 1, 2, 3, 4, 5]
# Use pd.cut to categorize the column values
filtered_df.loc[:,'trip_taker_household_income_bin'] = pd.cut(filtered_df['trip_taker_household_income'], bins=bin_edges, labels=bin_labels, right=False, ordered=False)#.fillna('9)')                            #change in .py NA-changes


# Checking
#filtered_df["trip_taker_household_income"].describe()        #has negative values too
#filtered_df["trip_taker_household_income"].info()            #Confirmed 0 empty
#df=filtered_df
#df[df["trip_taker_household_income_bin"].isnull()][['trip_taker_household_income_bin', 'trip_taker_household_income']].info()
#df[df["trip_taker_household_income_bin"].isnull()][['trip_taker_household_income_bin', 'trip_taker_household_income']].info()   664 rows

filtered_df["trip_taker_household_income_bin"].unique()

Subset_for_mapping = filtered_df.copy()
# To Map tracts to Travelshed                                                                                                                                                                                    2 Dict Vs Join
# Get the mapping file
TravelShed_w_all_FIPS = pd.read_csv(f"C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/TravelShed_FIPS.csv")
# This File is located in Sharepoint with block group code(bg_code) and its corresponding travelshed. Took out the last digit to get Tract FIPS and use it to map.   

# Put csv in the dictionary
TravelShed_dict = dict(zip(TravelShed_w_all_FIPS['tract'], TravelShed_w_all_FIPS['travelshed']))

#Mapping tract from original dataframe to Travelshed to new columns
Subset_for_mapping['Origin_sline_shed'] = Subset_for_mapping['origin_trct_fips_2020'].map(TravelShed_dict).fillna(99)#, inplace=True) 
Subset_for_mapping['Destination_sline_shed'] = Subset_for_mapping['destination_trct_fips_2020'].map(TravelShed_dict).fillna(99)#, inplace=True)
Subset_for_mapping['Trip_taker_home_sline_shed'] = Subset_for_mapping['trip_taker_home_trct_fips_2020'].map(TravelShed_dict).fillna(99)#, inplace=True)
Subset_for_mapping['Trip_taker_work_sline_shed'] = Subset_for_mapping['trip_taker_work_trct_fips_2020'].map(TravelShed_dict).fillna(99)#,inplace=True)

#Checking if location and TravelShed mapping went OK
#Subset_for_mapping[['Origin_sline_shed' ,'origin_trct_fips_2020', 'Destination_sline_shed' ,'destination_trct_fips_2020']].head(10)
#Subset_for_mapping[['Trip_taker_home_sline_shed' ,'trip_taker_home_trct_fips_2020', 'trip_taker_home_trct_2020', 'Trip_taker_work_sline_shed' ,'trip_taker_work_trct_fips_2020', 'trip_taker_work_trct_2020']].value_counts()

print("The shape after Travel Sheds is",  Subset_for_mapping.shape)           # 4 travel sheds: 'Origin_sline_shed', 'Destination_sline_shed','Trip_taker_home_sline_shed', 'Trip_taker_work_sline_shed'
                                                                              #+ 1 Income bin: 'trip_taker_household_income_bin'


print("The trip taker's home Travel-Shed are",Subset_for_mapping["Trip_taker_home_sline_shed"].value_counts())
print("The trip taker's home Travel-Shed are",Subset_for_mapping["Trip_taker_work_sline_shed"].value_counts())
print("The trip Origin Travel-Shed are",Subset_for_mapping["Origin_sline_shed"].value_counts())
print("The trip Destination Travel-Shed are",Subset_for_mapping["Destination_sline_shed"].value_counts())


#Subset_for_mapping.columns
#Subset_for_mapping.head(10)

subset_for_summary =Subset_for_mapping.copy()

# Start time >> Start hour
#subset_for_summary.rename(columns = {'trip_start_time':'trip_start_hour'}, inplace = True) 
# drop county code, trip duration, others
subset_for_summary = subset_for_summary.drop(['origin_cty_fips_2020','origin_trct_2020','destination_trct_2020', 'destination_cty_fips_2020','trip_taker_home_trct_2020','trip_taker_home_cty_2020', 'trip_duration_minutes'],axis=1) #,'trip_duration_minutes' #'origin_cty_fips_2020','trip_taker_work_trct_2020.1',
print("The shape is now", subset_for_summary.shape)
print("The columns remaining are", subset_for_summary.columns)

# Now categorizing these columns to numericals
#Primary-mode ['private_auto', 'auto_passenger', 'public_transit','other_travel_mode', 'on_demand_auto', 'biking', 'walking'], dtype=object)
# 1 'private_auto'
# 2  'auto_passenger'
# 3 'public_transit'
# 4 'on_demand_auto'
# 5 'biking'
# 6 'walking'
# 7 'other_travel_mode'

#Trip purpose 'home', 'work', 'shop', 'social', 'eat', 'other_activity_type', 'maintenance', 'school', 'recreation'
#1 'home'
#2 'work'
#3 'shop'
#4 'social'
#5 'eat'
#6 'school'
#7 'recreation'
#8 'maintenance'
#9 'other_activity_type'

#TravelShed

#Origin_sline_shed =           ['99', 'Tukwila', 'Auburn', 'Kent', 'Sumner', 'Lakewood', 'Puyallup', 'King Street', 'Tacoma Dome', 'South Tacoma'],dtype=object)
#'Destination_sline_shed'=     ['99', 'Kent', 'King Street', 'Tukwila', 'Puyallup', 'Sumner',  'Tacoma Dome', 'Auburn', 'Lakewood', 'South Tacoma'], dtype=object)
#'Trip_taker_home_sline_shed'= ['99', 'Kent', 'Tacoma Dome', 'Sumner', 'Lakewood', 'Puyallup','Auburn', 'King Street', 'Tukwila', 'South Tacoma'], dtype=object)
#'Trip_taker_work_sline_shed'= ['99', 'King Street', 'Tukwila', 'Kent', 'Auburn', 'Tacoma Dome', 'Puyallup', 'Sumner', 'Lakewood', 'South Tacoma'], dtype=object) 
#df['trip_purpose'].unique() 

# Categorizing a few columns to reduce the file size
# Creating an empty Dictionary
Dict = {}
Shed_list = [('Auburn',1), ('Kent',2), ('King Street',3), ('Lakewood',4), ('Puyallup',5), ('South Tacoma',6), ('Sumner',7), ('Tacoma Dome',8), ('Tukwila',9), (99,99)]
Mode_list = [('private_auto',1), ('auto_passenger',2), ('public_transit',3), ('on_demand_auto',4), ('biking',5), ('walking',6),('other_travel_mode',7)]
Trp_purpose_list= [('home',1), ('work',2), ('shop',3), ('social',4), ('eat',5), ('school',6), ('recreation',7), ('maintenance',8), ('other_activity_type',9)]
 
# Creating a Dictionary # with each item as a Pair
Shed_Dict = dict(Shed_list)
Mode_Dict = dict(Mode_list)
Trp_purpose_Dict= dict(Trp_purpose_list)
#print(Dict)


#Now Mapping
subset_for_summary['Origin_sline_shed_numeric'] = subset_for_summary['Origin_sline_shed'].map(Shed_Dict) 
subset_for_summary['Destination_sline_shed_numeric'] = subset_for_summary['Destination_sline_shed'].map(Shed_Dict) 
subset_for_summary['Trip_taker_home_sline_shed_numeric'] = subset_for_summary['Trip_taker_home_sline_shed'].map(Shed_Dict) 
subset_for_summary['Trip_taker_work_sline_shed_numeric'] = subset_for_summary['Trip_taker_work_sline_shed'].map(Shed_Dict) 

subset_for_summary['Primary_Mode_numeric'] = subset_for_summary['primary_mode'].map(Mode_Dict) 
subset_for_summary['Trip_purpose_numeric'] = subset_for_summary['trip_purpose'].map(Trp_purpose_Dict)

# checking the map 
subset_for_summary [['Origin_sline_shed','Origin_sline_shed_numeric','Destination_sline_shed', 'Destination_sline_shed_numeric','Trip_taker_home_sline_shed','Trip_taker_home_sline_shed_numeric','Trip_taker_work_sline_shed','Trip_taker_work_sline_shed_numeric','primary_mode','Primary_Mode_numeric','trip_purpose','Trip_purpose_numeric']]

subset_for_summary[['Origin_sline_shed','Origin_sline_shed_numeric','Destination_sline_shed', 'Destination_sline_shed_numeric','Trip_taker_home_sline_shed','Trip_taker_home_sline_shed_numeric','Trip_taker_work_sline_shed','Trip_taker_work_sline_shed_numeric','primary_mode','Primary_Mode_numeric','trip_purpose','Trip_purpose_numeric']].tail(10)
print(subset_for_summary["Origin_sline_shed"].value_counts())

#Now, trying to get only HBW trips & its hourly counts
#For HBW Columns
HBW_col = ['trip_purpose','previous_trip_purpose' ]

# Concatenate values of all columns to create a new column 'concatenated'
subset_for_summary['HBW_concat'] = subset_for_summary[HBW_col].agg('-'.join, axis=1)

subset_for_summary[['trip_purpose','previous_trip_purpose','HBW_concat','trip_start_time']]
subset_for_summary =  subset_for_summary[subset_for_summary['HBW_concat'].str.contains('work-home|home-work')] 
print("The HBW trips table's size is", subset_for_summary.shape)




file_name = os.path.basename(file_path).replace(".csv", "_afterNumeric.csv")
file_path = os.path.join("C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final", file_name).replace("\\","/")
subset_for_summary.to_csv(file_path, index=False)

#Since mapping is ok
drop_col = ['trip_taker_work_cty_2020','Origin_sline_shed','Destination_sline_shed', 'Trip_taker_home_sline_shed','Trip_taker_work_sline_shed','primary_mode', 'trip_purpose']
subset_after_numeric = subset_for_summary.drop(columns=drop_col, axis=1)

#Lets rename the numeric columns & others


subset_after_numeric = subset_after_numeric.rename(columns={'Origin_sline_shed_numeric': 'origin_travel_shed', 
                        'Destination_sline_shed_numeric': 'destination_travel_shed',
                        'Trip_taker_home_sline_shed_numeric':'trip_taker_home_travel_shed',
                        'Trip_taker_work_sline_shed_numeric': 'trip_taker_work_travel_shed',
                        'Primary_Mode_numeric':'primary_mode',
                        'Trip_purpose_numeric':'trip_purpose',
                        'trip_start_time':'trip_start_hour'
                        })

print("Now the shape is", subset_after_numeric.shape)
print("The columns are", subset_after_numeric.columns)

final_subset = subset_after_numeric[['origin_trct_fips_2020','destination_trct_fips_2020','primary_mode', 'trip_purpose','previous_trip_purpose','trip_start_hour','trip_distance_miles','trip_taker_work_trct_fips_2020','trip_taker_home_trct_fips_2020','trip_taker_household_income_bin','origin_travel_shed',
       'destination_travel_shed', 'trip_taker_home_travel_shed','trip_taker_work_travel_shed', 'HBW_concat' ]]

#final_subset
# Concatenate values of all columns to create a new column 'concatenated'
final_subset['concatenated'] = final_subset[final_subset.columns].astype(str).agg('-'.join, axis=1)

# Create a new column 'counts' by counting occurrences of each unique combination
final_subset['trip_counts'] = final_subset.groupby('concatenated')['concatenated'].transform('count')        # I could use trip counts

# Drop the 'concatenated' column if not needed anymore
final_subset = final_subset.drop(columns=['concatenated'])
#final_subset

#hourly_counts=final_subset.groupby('trip_start_hour').size()

#To minimize the file size # I should have done much earlier
cols_toInt8 = ['primary_mode', 'trip_purpose','trip_start_hour','trip_distance_miles','origin_travel_shed','destination_travel_shed','trip_taker_home_travel_shed','trip_taker_work_travel_shed','trip_counts']
final_subset[cols_toInt8] = final_subset[cols_toInt8].applymap(np.int8)

# Writing final_subset
#Subset_before_merged = "C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset"
#file_path = # already given above while reading original replica download
file_name = os.path.basename(file_path).replace("_afterNumeric.csv", "_subset.csv")
file_path = os.path.join("C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/Final_Subset", file_name).replace("\\","/")
final_subset.to_csv(file_path, index=False)



#Sample
file_name = os.path.basename(file_path).replace( "_afterNumeric.csv", ".csv")
path=os.path.join("C:/Users/USAP093587/OneDrive - WSP O365/Seattle_Transit_Project/Replica_downloads/Subset/Final/Sample_final","_FinalSubset_Sample.csv").replace("\\","/")
final_subset.sample(100).to_csv(path, index=False)

In [ ]:
# Checking trips between travelsheds

trips_within_travelshed = final_subset[(final_subset['origin_travel_shed'].isin([1,2,3,4,5,6,7,8,9])) &              #99 not in travel shed
                                             (final_subset['destination_travel_shed'].isin([1,2,3,4,5,6,7,8,9]))]

hourly_counts=trips_within_travelshed.groupby('trip_start_hour').size()

import matplotlib.pyplot as plt
# Create a histogram
plt.figure(figsize=(10, 6))  # Optional: Set the figure size
plt.bar(hourly_counts.index, hourly_counts, color='skyblue', edgecolor='black')
plt.xlabel('Hour of the Day')
plt.ylabel('Count')
plt.title('Hourly Count of Trips')
plt.grid(axis='y')  # Optional: Add grid lines

# Show the plot
plt.show()

